In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [8]:
import pandas as pd
from os import listdir
from os.path import isfile, join
import json
import tarfile
import pickle
import matplotlib.pyplot as plt
import scipy.io.wavfile
import scipy.signal
from moviepy.editor import *
from tqdm import tqdm

In [3]:
def function_reu(x):
  try:
    return dic[x]
  except:
    return ' '
def function_test(x):
  try:
    return dictionnary[(x+ '.mp3')]
  except:
    return 'do not have'

In [4]:
file = open('/content/gdrive/MyDrive/Projet_Multimedia/download/Texte/trans0.txt', encoding='utf-16')
text0 = file.read()
d = json.loads(text0)
file = open('/content/gdrive/MyDrive/Projet_Multimedia/download/Texte/trans1.txt', encoding='utf-16')
text1 = file.read()
d1 = json.loads(text1)
file = open('/content/gdrive/MyDrive/Projet_Multimedia/download/Texte/trans2.txt', encoding='utf-16')
text2 = file.read()
d2 = json.loads(text2)
file = open('/content/gdrive/MyDrive/Projet_Multimedia/download/Texte/trans_rest.txt', encoding='utf-16')
text3 = file.read()
d3 = json.loads(text3)
dictionnary = d | d1
dictionnary = dictionnary | d2
dictionnary = dictionnary | d3
len(dictionnary)
descr = pd.read_csv('/content/gdrive/MyDrive/Projet_Multimedia/download/CSV/Transcription_wav.csv')
name_video = descr['name_vid']
contenue = descr['contenue']
dic = {}
vid_name_set = set()
for vid_name, cont in zip(list(name_video), list(contenue)):
  if not 'artificial intelligence' in cont.lower():
    if not vid_name in dic.keys():
      dic[vid_name] = cont
    else:
      dic[vid_name] += cont
  else:
    vid_name_set.add(vid_name)
    print(vid_name)
descr['descr'] = descr['name_vid'].apply(lambda x : function_reu(x))
descr = descr.drop_duplicates(subset='name_vid')
dt = pd.read_csv('/content/gdrive/MyDrive/Projet_Multimedia/download/CSV/LVT.csv')
dt['name'] = dt['video'].apply(lambda x : x[5:-4])
dt['transcription'] = dt['name'].apply(lambda x : function_test(x))
dt = dt.drop('video', axis=1)
dt = dt[dt.transcription != 'do not have']
intersection = pd.merge(dt, descr, left_on='name', right_on='name_vid')
intersection = intersection.drop('label', axis = 1)
intersection = intersection.drop('Unnamed: 0', axis = 1)
intersection = intersection.drop('name_audio', axis = 1)
intersection = intersection.drop('id_audio', axis = 1)
intersection = intersection.drop('name_vid', axis = 1)
intersection = intersection.drop('contenue', axis = 1)
intersection['total'] = intersection['text'] + intersection['descr'] + intersection['transcription']
print(len(intersection), len(dt), len(descr))

-eOrNdBpGMv8
1551 2085 1878


In [5]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 67.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 84.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.9 MB/s eta 0:00:00


In [6]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

In [10]:
  # Load model from HuggingFace Hub
  tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-mpnet-base-v2')
  model = AutoModel.from_pretrained('sentence-transformers/all-mpnet-base-v2')

In [11]:
list_sentence_embeddings = []
for count in tqdm(range(21)):
  # Sentences we want sentence embeddings for
  sentences = dt['text']

  # Tokenize sentences
  try:
    encoded_input = tokenizer(list(sentences)[(100*count):(100*(count+1))], padding=True, truncation=True, return_tensors='pt')
  except:
    encoded_input = tokenizer(list(sentences)[(100*count):-1], padding=True, truncation=True, return_tensors='pt')

  # Compute token embeddings
  with torch.no_grad():
      model_output = model(**encoded_input)

  # Perform pooling
  sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

  # Normalize embeddings
  list_sentence_embeddings.append(F.normalize(sentence_embeddings, p=2, dim=1))

100%|██████████| 21/21 [54:54<00:00, 156.90s/it]


In [ ]:
len(list_sentence_embeddings)

23

In [ ]:
type(list_sentence_embeddings[0])

torch.Tensor

In [12]:
emb = torch.Tensor()
for i in range(21):
  emb = torch.concat([emb,list_sentence_embeddings[i]], 0)
emb.shape

torch.Size([2085, 768])

In [15]:
import numpy as np

In [17]:
np.save('/content/gdrive/MyDrive/Projet_Multimedia/download/Embeddings/Texte_Embeddings/mp.npy', emb.numpy())

In [ ]:
np.save('/content/gdrive/MyDrive/Projet_Multimedia/download/Embeddings/Texte_Embeddings/MPnet_Desc_ecrit.npy', emb.numpy())

In [ ]:
import numpy as np

In [ ]:
to_data_frame_emb = []
for i in range(len(emb)):
  to_data_frame_emb.append(emb[i,:])

In [ ]:
df['emb'] = to_data_frame_emb

In [ ]:
df

,name,label,duree,transcription,emb
0,-KHQk1_Vq69E,Autos & Vehicles,21,"As you can see, the employment rate for Ameri...","[tensor(-0.0784), tensor(0.1044), tensor(-0.03..."
1,-mG4Y2Snygfk,People & Blogs,60,So So you,"[tensor(-0.0409), tensor(0.0056), tensor(-0.01..."
2,-OdEfx8up7wM,People & Blogs,69,I want to hear a manly rip. Show us what you ...,"[tensor(0.0240), tensor(0.0123), tensor(0.0127..."
3,-Opbfh4wNu7Q,Entertainment,61,If we do this we'd be going in shorthanded. Y...,"[tensor(0.0005), tensor(0.0674), tensor(0.0094..."
4,-A5340xbbQJU,Sports,299,This is the front flip shot. Got a chance peo...,"[tensor(0.0030), tensor(-0.0464), tensor(0.016..."
...,...,...,...,...,...
2275,-ymc30meWzfg,Education,79,My name is Brady Anderson. I'm a graduate tea...,"[tensor(0.0271), tensor(0.0194), tensor(0.0018..."
2276,-h7HGWRTGqyg,Education,164,I'm gonna staple some more. There we go. Good...,"[tensor(0.0247), tensor(-0.0177), tensor(0.006..."
2277,-XVRLTfN1WYw,Howto & Style,90,"Music you Oh, yes!","[tensor(6.5490e-05), tensor(0.0511), tensor(-0..."
2278,-jTtH7Fq1trw,Travel & Events,185,It's a great sampler of small to small to sma...,"[tensor(0.0214), tensor(-0.0193), tensor(-0.01..."


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
df['label'] = df['label'].astype('category')

# Séparer les données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(emb, df['label'], test_size=0.15, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15, random_state=1)

# Encoder les textes avec Sentence-Transformers
train_embeddings = X_train
test_embeddings  = X_test
valid_embeddings  = X_val

# Définir le modèle de classification
class ClassificationModel(torch.nn.Module):
    def __init__(self, input_size, num_labels):
        super(ClassificationModel, self).__init__()
        self.fc = torch.nn.Linear(input_size, 2048)
        self.r = torch.nn.ReLU()
        self.m = torch.nn.Dropout(p=0.5)
        self.fc2 = torch.nn.Linear(2048, 1024)
        self.fc3 = torch.nn.Linear(128, num_labels)

    def forward(self, x):
      x = self.fc(x)
      x = self.r(x)
      x = self.m(x)
      x = self.fc2(x)
      x = self.r(x)
      x = self.m(x)
      x = self.fc3(x)
      x = self.r(x)
      return x

# Initialiser le modèle de classification
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
model = ClassificationModel(input_size=train_embeddings.size(-1), num_labels=len(df['label'].cat.categories)).to(device)

# Définir le DataLoader
batch_size   = 8
train_loader = DataLoader(TensorDataset(train_embeddings.to(device), torch.tensor(y_train.cat.codes.tolist(), dtype=torch.long).to(device)), batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(TensorDataset(test_embeddings.to(device) , torch.tensor(y_test.cat.codes.tolist(), dtype=torch.long).to(device)), batch_size=batch_size, shuffle=False)
valid_loader  = DataLoader(TensorDataset(valid_embeddings.to(device) , torch.tensor(y_val.cat.codes.tolist(), dtype=torch.long).to(device)), batch_size=batch_size, shuffle=False)

# Définir l'optimiseur
optimizer = torch.optim.Adam(model.parameters())

num_epochs = 10

# Entraîner le modèle
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    total_loss_valid = 0.0

    for batch in train_loader:
        embeddings, labels = batch
        optimizer.zero_grad()
        outputs = model(embeddings)
        loss = torch.nn.functional.cross_entropy(outputs, labels)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

    average_loss = total_loss / len(train_loader)
    for batch in valid_loader:
        embeddings, labels = batch
        optimizer.zero_grad()
        outputs = model(embeddings)
        loss = torch.nn.functional.cross_entropy(outputs, labels)
        total_loss_valid += loss.item()
    average_loss_valid = total_loss_valid / len(valid_loader)
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {average_loss}, Val_loss: {average_loss_valid}')

# Évaluer le modèle
model.eval()
all_preds  = []
all_labels = []

with torch.no_grad():
    for batch in test_loader:
        embeddings, labels = batch
        outputs = model(embeddings)
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.tolist())
        all_labels.extend(labels.tolist())

# Calculer la précision
accuracy = accuracy_score(all_labels, all_preds)
print(f'Accuracy: {accuracy}')

# Sauvegarder le modèle
torch.save(model.state_dict(), 'Model_with_sentence_transformers.pth')

RuntimeError: ignored

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [ ]:
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(X_train,y_train)
tfsvmpred = clf.predict(X_test)
accuracy_score(y_test,tfsvmpred)

In [ ]:
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(X_train,y_train)
tfsvmpred = clf.predict(X_val)
accuracy_score(y_val,tfsvmpred)

In [ ]:
predovr = OneVsRestClassifier(SVC(random_state=0)).fit(X_train, y_train).predict(X_test)
accuracy_score(y_test,predovr)

In [ ]:
predovr = OneVsRestClassifier(SVC(random_state=0)).fit(X_train, y_train).predict(X_val)
accuracy_score(y_val,predovr)

In [ ]:
sentence_embeddings.shape

In [ ]:
sentence_embeddings.shape